<a href="https://colab.research.google.com/github/Stalker-404/Whut_electric/blob/main/Whut_electric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安装依赖环境

In [ ]:
!pip3 install selenium
!apt-get update 
!apt install chromium-chromedriver

# 填入个人信息参数

In [ ]:
# 填入个人学号
usercode = "Your ID Code"
# 填入缴费平台密码
passwd = "Your Password"
# 校区
area = "Your Area"
# 楼栋
building = "Your Building"
# 楼层
floor = "Your Floor"
# 房间
room = "Your Room"
# 当前电价
price = 0.59
# 填入通过Serverchan获取的sendkey
key = "**********************************"

# 爬取剩余电费和电量

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import re
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--headless')

driver = webdriver.Chrome(chrome_options=chrome_options)

try:
  #连接到登陆平台
  driver.get('http://cwsf.whut.edu.cn/MNetWorkUI/slogin.html')

  #忽略2022年3月25日开始出现的的弹窗信息
  driver.implicitly_wait(20)
  jumpalert = driver.switch_to.alert
  jumpalert.accept() 

  #选择平台账号登陆
  select_logintype = Select(driver.find_element_by_id('logintype'))
  select_logintype.select_by_value('PLATFORM')
  driver.implicitly_wait(20)

  # 填写账号密码
  input_account = driver.find_element_by_id('usercode')
  input_account.send_keys(usercode)
  input_password = driver.find_element_by_id('passwd')
  input_password.send_keys(passwd)

  # 账号登陆
  account_login_button = driver.find_element_by_xpath('//div[@class="change-list-4-clearfix"]/input[1]')
  account_login_button .click()

  # 进入缴费平台
  driver.implicitly_wait(20)
  click3_button = driver.find_element_by_link_text('马房山学生电费')
  click3_button.click()

  # 填写数据
  select1 = Select(driver.find_element_by_id('areaid'))
  select1.select_by_visible_text(area)
  driver.implicitly_wait(20)
  select2 = Select(driver.find_element_by_id('building'))
  select2.select_by_visible_text(building)
  driver.implicitly_wait(20)
  select4 = Select(driver.find_element_by_id('floorid'))
  select4.select_by_visible_text(floor)
  driver.implicitly_wait(20)
  select5 = Select(driver.find_element_by_id('roomid'))
  select5.select_by_visible_text(room)

  # 读取剩余电量字符串，并按小数点分割整数和小数部分后转换为数值量
  result = driver.find_element_by_id('MeterOverdue').text
  balance = re.findall(r"\d+\.?\d*",result)
  balance = float(''.join(balance))
  # 按照当前电价计算剩余度数，保留两位小数
  power = round((balance / price),2)    

  # 结束运行
  errstatus = 0
  driver.quit()
 
except Exception as e:
  
  #结束运行
  errstatus = 1
  driver.quit()

  #保存错误信息
  errinfo = e

# 调用ServerChan进行消息推送

In [ ]:
import requests
from urllib.parse import quote

if errstatus == 0:
  title = "{}电费剩余{}元（{}度）".format(room , str(balance) , str(power)) #标题
  content = """
  **所在校区：**{}\n
  **所在楼栋：**{}\n
  **所在楼层：**{}\n
  **房间号码：**{}
  """.format(area , building , floor , room) #信息内容

else:
  title = "电费查询脚本出错！"
  content = """
  **所在校区：**{}\n
  **所在楼栋：**{}\n
  **所在楼层：**{}\n
  **房间号码：**{}\n
  **报错信息：** \n
  {}
  """.format(area , building , floor , room , errinfo) #信息内容

# 进行URL转义
title = quote(title, 'utf-8') #对信息标题进行URL转义
content = quote(content, 'utf-8') #对信息内容进行URL转义

# 发送信息
requests.get("https://sc.ftqq.com/{}.send?text={}&desp={}".format(key,title,content))